In [2]:
import os
import fitz

In [3]:
MIN_WIDTH, MIN_HEIGHT = 100, 100
MAX_WIDTH, MAX_HEIGHT = 5000, 5000

In [4]:
pdf_file = fitz.open("./hdsh nhập điểm.pdf")
output_dir = "./output"
if not os.path.exists(output_dir): os.makedirs(output_dir)

for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    image_list = page.get_images(full=True)
    print(f"Trang {page_index+1} có {len(image_list)} ảnh.")
    for img_index, img in enumerate(image_list):
        xref = img[0]
        base_image = pdf_file.extract_image(xref)
        
        # Lấy thông số kích thước
        width = base_image["width"]
        height = base_image["height"]
        
        # KIỂM TRA NGƯỠNG KÍCH THƯỚC
        if (MIN_WIDTH <= width <= MAX_WIDTH) and (MIN_HEIGHT <= height <= MAX_HEIGHT):
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            
            image_name = f"image_p{page_index+1}_{img_index+1}.{image_ext}"
            image_path = os.path.join(output_dir, image_name)
            
            with open(image_path, "wb") as f:
                f.write(image_bytes)
            print(f"Đã lưu: {image_name} ({width}x{height})")
        else:
            print(f"Bỏ qua ảnh {width}x{height} do không nằm trong ngưỡng.")

pdf_file.close()

Trang 1 có 1 ảnh.
Đã lưu: image_p1_1.png (645x793)
Trang 2 có 1 ảnh.
Đã lưu: image_p2_1.png (670x298)
Trang 3 có 3 ảnh.
Đã lưu: image_p3_1.png (678x280)
Đã lưu: image_p3_2.png (675x285)
Đã lưu: image_p3_3.png (653x175)
Trang 4 có 1 ảnh.
Đã lưu: image_p4_1.png (661x300)
